In [128]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
import numpy as np

In [129]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = "/kaggle/input/pizza-classification-data/train"  # e.g. './muffin-vs-chihuahua/train'
test_dir = "/kaggle/input/pizza-classification-data/test"    # e.g. './muffin-vs-chihuahua/test'

In [130]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [131]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# Load ResNet50 with pretrained ImageNet weights
base_model = ResNet50(
    weights='/kaggle/input/resnet50-weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
    include_top=False,  # Exclude final classification layers
    input_shape=(128, 128, 3)
)

# Freeze the base model
base_model.trainable = False

# Build custom classification head
model_resnet = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

# Compile with lower learning rate for transfer learning




Found 1280 images belonging to 2 classes.
Found 320 images belonging to 2 classes.
Found 366 images belonging to 2 classes.


In [132]:
# Configure the model optimizers, loss function, and metrics
model_resnet.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model_resnet.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 4, 4, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,112,513 (91.98 MB)

 Trainable params: 524,801 (2.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [133]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 28s 703ms/step - accuracy: 0.7290 - loss: 0.5402 - val_accuracy: 0.7094 - val_loss: 0.5887
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 28s 685ms/step - accuracy: 0.7309 - loss: 0.5325 - val_accuracy: 0.6031 - val_loss: 0.7347
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 28s 702ms/step - accuracy: 0.7604 - loss: 0.5070 - val_accuracy: 0.8031 - val_loss: 0.5024
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 28s 687ms/step - accuracy: 0.7440 - loss: 0.5284 - val_accuracy: 0.6187 - val_loss: 0.6320
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 41s 695ms/step - accuracy: 0.7750 - loss: 0.4834 - val_accuracy: 0.6719 - val_loss: 0.5990
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 27s 681ms/step - accuracy: 0.7707 - loss: 0.5088 - val_accuracy: 0.7844 - val_loss: 0.4816
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 42s 711ms/step - accuracy: 0.7732 - loss: 0.4655 - val_accuracy: 0.7812 - val_loss: 0.4716
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 28s 687ms/step - accuracy: 0.7818 - loss: 0.4489 - val_accu

In [134]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 188ms/step - accuracy: 0.6422 - loss: 0.7034
Test Accuracy: 0.7404371500015259


In [135]:
# SAVE THE MODEL
model.save('/kaggle/working/models/exercise_6_custom_tambalo.h5')

In [136]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='/kaggle/working/models/exercise_6_custom_tambalo.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Pizza" if pred >= 0.5 else "Not Pizza"
    print(f"Prediction: {label} (confidence: {pred:.2f})")
    return pred


In [137]:
# Example usage:
pred1 = predict_image("/kaggle/input/pizza-classification-data/test/not_pizza/3140015.jpg")
pred2 = predict_image("/kaggle/input/pizza-classification-data/test/not_pizza/3194027.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step


Prediction: Not Pizza (confidence: 0.49)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
Prediction: Not Pizza (confidence: 0.41)
